In [1]:
import numpy as np
import sys
import time
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from IPython.core.display import display, HTML

sys.path.append(r"C:\Users\hyli\Documents\GitHub\CrystalDiff")
display(HTML("<style>.container { width:90% !important; }</style>"))

from CrystalDiff import util, pulse, lclsutil, groutine

In [3]:
number_x = 2
number_y = 500
number_z = 10 ** 5
kx_grid, ky_grid, kz_grid, axis_info = lclsutil.get_k_mesh_3d(number_x=number_x,
                                                              number_y=number_y,
                                                              number_z=number_z,
                                                              delta_e_x=1e-50,
                                                              delta_e_y=5e-4,
                                                              delta_e_z=1e-3 / util.c)

# Apply fft shift
kx_grid = np.ascontiguousarray(np.fft.fftshift(kx_grid))
ky_grid = np.ascontiguousarray(np.fft.fftshift(ky_grid))
kz_grid = np.ascontiguousarray(np.fft.fftshift(kz_grid))

In [9]:
def Complex2HSV(z, rmin, rmax, hue_start=90):
    # get amplidude of z and limit to [rmin, rmax]
    amp = np.abs(z)
    amp = np.where(amp < rmin, rmin, amp)
    amp = np.where(amp > rmax, rmax, amp)
    
    #amp = rmax - amp
    
    ph = np.angle(z, deg=1) + hue_start
    # HSV are values in range [0,1]
    h = (ph % 360) / 360
    s = np.ones_like(h)*0.9
    v = (amp -rmin) / (rmax - rmin)
    return hsv_to_rgb(np.dstack((h,s,v)))

In [37]:
z_idx_range=500
z_start = 200
x_start = 100


for l in [2, 4, 8, 16, 24, 32]:
    # Load data 
    field = np.load(r"C:/Users/hyli/Desktop/Research/Report/2019-8-8/Data/multiple_reflections_{}_.npy".format(l))
    field = field[x_start:, z_start:]
    amplitude = np.abs(field)
    angle = np.angle(field)
    
    # Plot the magnitude
    fig, axes = plt.subplots(nrows=1, ncols=1)
    fig.set_figheight(4)
    fig.set_figwidth(6)


    im = axes.imshow(np.square(amplitude),
                     #vmin= 0.,
                     #vmax = 12.,
                     cmap='jet', 
                     origin='lower')

    axes.set_title("Intensity", fontsize=15)
    axes.set_yticks(np.arange(number_y - x_start, step=50))
    axes.set_yticklabels(axis_info["y_idx_tick"][x_start::50])
    axes.set_xticks(np.arange(z_idx_range - z_start, step=80))
    axes.set_xticklabels(axis_info["z_idx_tick"][number_z//2 :
                                                  number_z//2 + z_idx_range - z_start:80])

    axes.set_xlabel("Propagation direction (um)")
    axes.set_ylabel("Transversal direction (fs)")
    fig.colorbar(im, ax=axes)
    
    plt.savefig("C:/Users/hyli/Desktop/intensity_{}.png".format(l), bbox_inches='tight', transparent=True)
    plt.close(fig)
    
    
    # Plot the phase
    fig, axes = plt.subplots(nrows=1, ncols=1)
    fig.set_figheight(4)
    fig.set_figwidth(6)

    im = axes.imshow(angle,
                     #vmin= 0.,
                     #vmax = 12.,
                     cmap='jet', 
                     origin='lower')

    axes.set_title("Total Envelope Phase", fontsize=15)
    axes.set_yticks(np.arange(number_y - x_start, step=50))
    axes.set_yticklabels(axis_info["y_idx_tick"][x_start::50])
    axes.set_xticks(np.arange(z_idx_range - z_start, step=80))
    axes.set_xticklabels(axis_info["z_idx_tick"][number_z//2 :
                                                  number_z//2 + z_idx_range - z_start:80])


    axes.set_xlabel("Propagation direction (um)")
    axes.set_ylabel("Transversal direction (fs)")
    fig.colorbar(im, ax=axes)
    # Show the canvas
    plt.savefig("C:/Users/hyli/Desktop/phase_{}.png".format(l), bbox_inches='tight', transparent=True)
    plt.close(fig)

In [33]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb


def Complex2HSV(z, rmin, rmax, hue_start=90):
    # get amplidude of z and limit to [rmin, rmax]
    amp = np.abs(z)
    amp = np.where(amp < rmin, rmin, amp)
    amp = np.where(amp > rmax, rmax, amp)
    
    #amp = rmax - amp
    
    ph = np.angle(z, deg=1) + hue_start
    # HSV are values in range [0,1]
    h = (ph % 360) / 360
    s = np.ones_like(h)*0.9
    v = (np.exp((amp -rmin) / (rmax - rmin)) - 1.) / (np.e - 1)
    return hsv_to_rgb(np.dstack((h,s,v)))


data = np.load("C:/Users/hyli/Desktop/Research/Report/2019-8-8/Data/multiple_reflections_8_.npy")


linear = -45.1 + .107 * np.arange(data.shape[1])

new_data_2 = np.cos(linear) + 1.j * np.sin(linear)
new_data_2 = new_data_2[np.newaxis,:]

new_data_3 = data/new_data_2

image = Complex2HSV(z=new_data_3[100:, 200:], rmin=0, rmax=50, hue_start=90)

# Plot the phase
fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_figheight(4)
fig.set_figwidth(6)

im = axes.imshow(1-image,
                 #vmin= 0.,
                 #vmax = 12.,
                 cmap='jet', 
                 origin='lower')

axes.set_title("Complex Field", fontsize=15)
axes.set_yticks(np.arange(number_y - x_start, step=50))
axes.set_yticklabels(axis_info["y_idx_tick"][x_start::50])
axes.set_xticks(np.arange(z_idx_range - z_start, step=80))
axes.set_xticklabels(axis_info["z_idx_tick"][number_z//2 :
                                              number_z//2 + z_idx_range - z_start:80])


axes.set_xlabel("Propagation direction (um)")
axes.set_ylabel("Transversal direction (fs)")
fig.colorbar(im, ax=axes)
# Show the canvas
plt.savefig("C:/Users/hyli/Desktop/complex_field_{}.png".format(l), bbox_inches='tight', transparent=True)
plt.close(fig)